<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> 
        <h1>Natural Language Processing</h1>
        <h1>Applications</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
        @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import string
import gzip
from collections import Counter
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pprint
from pprint import pprint

import watermark

%matplotlib inline
%load_ext watermark

List out the versions of all loaded libraries

In [2]:
%watermark -n -v -m -g -iv

Set the default style

In [3]:
plt.style.use('./d4sci.mplstyle')

## word2vec embeddings

We start by loading a well trained set of word embeddings from project polyglot: https://sites.google.com/site/rmyeid/projects/polyglot


In [4]:
word_list, embeddings = pd.read_pickle('data/polyglot-en.pkl')
embeddings = normalize(embeddings)
word_list = np.array(word_list)
word_dict = dict(zip(word_list, range(embeddings.shape[0])))
print(embeddings.shape)

## Find the most similar words

In [5]:
def most_similar(word, embeddings, dictionary, reverse_dictionary, top_k=8):   
    valid_word = dictionary[word]
    similarity = cosine_similarity(embeddings, embeddings[valid_word, :].reshape(1, -1))
    nearest = (-similarity).argsort(axis=0)[1:top_k + 1].flatten()
    return reverse_dictionary[nearest]

In [6]:
most_similar("King", embeddings, word_dict, word_list)

## Analogies

Question task set downloaded from: http://download.tensorflow.org/data/questions-words.txt

In [7]:
questions = pd.read_csv('data/questions-words.txt', comment=':', sep=' ', header=None)
print(questions.shape)

In [8]:
questions.head()

Let us now define a function to automatically evaluate this specific type of analogy. We simply look up the embeddings for each of the four words in the question and perform the necessary vector algebra. To be safe, we enclose the entire function into a try/except block to catch the exceptions thrown when we try to use a word that is not part of our vocabulary (included in the embeddings)

In [9]:
def evaluate_analogy(question):
    word1, word2, word3, word4 = question

    if word1 not in word_dict or \
       word2 not in word_dict or \
       word3 not in word_dict or \
       word4 not in word_dict:
        return None

    key1 = word_dict[word1]
    key2 = word_dict[word2]
    key3 = word_dict[word3]
    key4 = word_dict[word4]

    vec1 = embeddings[key1, :]
    vec2 = embeddings[key2, :]
    vec3 = embeddings[key3, :]
    vec4 = embeddings[key4, :]

    predict = vec2-vec1+vec3

    sim = np.matmul(predict, embeddings.T)
    nearest = np.argsort(-sim)[:10]

    return word4 in word_list[nearest]

In [10]:
evaluate_analogy(questions.iloc[1])

In [11]:
questions.iloc[1]

In [12]:
results = [evaluate_analogy(questions.iloc[i]) for i in range(1000)]

In [13]:
clean_results = [res for res in results if res is not None]
accuracy = np.mean(clean_results)
print(accuracy)

We this simple approach we achieve ~53% accuracy. Our results are penalized by the fact that our embeddings weren't generated specifically for this purpose and are missing some of the words used in the analogies.

## Visualization

In [14]:
plt.figure(figsize=(15, 5))
plt.imshow(embeddings.T, aspect=700, cmap=cm.seismic)
plt.grid(None)
plt.xlabel("vocabulary")
plt.ylabel("dimensions")
plt.colorbar(orientation='vertical')

## Visualizing the embedding space

Using a dimensionality reduction algorithm like t-SNE we are able to project our word embeddings into a two dimensional space for visualization purposes. While the details of how t-SNE does its magic are beyond the scope of this course, the fundamental idea is the same of algorithms like PCA or the matrix decomposition methods we explored. Project the dataset into a latent, and lower dimensional, space in a smart way. t-SNE became popular in recent year due to its hability to make projections into 2D space in a way that preserves as much of the higher dimensional structure as possible resulting in beautiful and useful visualizations. 

The start implementation of t-SNE is that that comes bundled with **sklearn**. Here we simply call it to generate the plot we saw in the slides.

In [15]:
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
plot_only = 500 # Plot only 500 words
low_dim_embs = tsne.fit_transform(np.array(embeddings)[:plot_only, :])
labels = [word_list[i] for i in range(plot_only)]

In [16]:
plt.figure(figsize=(18, 18))

for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                fontsize=12)

## Word co-occurences

GloVe relies on the word co-occurences matrix. Let's take a look in detail on how to calculate it. For convenience, we'll use the same nursery rhyme we looked at in Lesson I. To save time we'll just load the data from a file

In [17]:
data = pickle.load(open("data/mary.pickle", "rb"))
mary_word_list = data['word_list']
mary_word_dict = data['word_dict']
text_words = data['text_words']

Let's check that everything looks ok

In [18]:
for word in mary_word_dict:
    if mary_word_list[mary_word_dict[word]] != word:
        print("ERROR!")

And that we got the right text

In [19]:
print(text_words)

And now we're ready to define a function that calculates the cooccurence matrix

In [20]:
def cooccurence_matrix(word_dict, text_words, window_size=1):
    vocabulary_size = len(word_dict)
    matrix = np.zeros((vocabulary_size, vocabulary_size), dtype='int')

    for i in range(window_size+1, len(text_words)-window_size):
        word_id = word_dict[text_words[i]]
        
        for j in range(i-window_size, i+window_size+1):
            if j == i: 
                continue
            
            context_id = word_dict[text_words[j]]
            
            matrix[word_id, context_id] += 1

    return matrix

Let's take a look at what this matrix looks like

In [21]:
matrix = cooccurence_matrix(mary_word_dict, text_words)

Visualize the matrix elements

In [22]:
temp = matrix.astype('str')
temp[temp=='0'] = ""
pd.DataFrame(temp)

While it might appear symmetrical, that is actually not true, as we can easily see:

In [23]:
np.sum(np.abs(matrix-matrix.T))

or more directly

In [24]:
print(matrix[mary_word_dict['a'], mary_word_dict['had']])
print(matrix[mary_word_dict['had'], mary_word_dict['a']])

The most common implementations of GloVe scale the contribution of each context word by its distance to the center word. In this way, nearer words contribute more than more distant ones. Using this unweighted definition, we can also calculate the conditional probability $P(w|C)$:

In [25]:
Prob = matrix/matrix.sum(axis=0)

A simple example will confirm that this is correct:

In [26]:
Prob[mary_word_dict['mary'], mary_word_dict['had']]

This implies that 1/3. of the occurences of the word mary are next to the word 'had'. Let's confirm it explicitly

In [27]:
for i in range(len(text_words)):
    if text_words[i] == 'mary':
        if i != 0:
            print(text_words[i-1], text_words[i], text_words[i+1])
        else:
            print(None, text_words[i], text_words[i+1])

Naturally, the complementary probability $P(had|mary)$ is different

In [28]:
Prob[mary_word_dict['had'], mary_word_dict['mary']]

As we saw, GloVe relies on this coocurrence matrix to define its embeddings. We can also easily see how more complex language models of the form $P(word|word_1,word_2, \cdots)$ can be obtained by changing the way in which the columns are defined (and increasing their number significantly). 

Surprisingly, in 2014, it was shown by O. Levy and Y. Goldberg in a highly cited [NIPS paper](https://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization) that some variations of word2vec are equivalent to factorizing a word-context matrix. These two approches for obtaining word embeddings aren't so differnet after all.


## Language Detection

We start by building a vector of character distributions for each language. Due to the total size of the google books dataset, we include only a partial file in the data directory. The interested student is encouraged to download all teh files and use the code below to build hers or his own language detector.

In [29]:
characters = sorted(set(string.ascii_letters.lower()))

dict_char = dict(zip(characters, range(len(characters))))
counts = np.zeros(len(characters), dtype='uint64')

line_count = 0

filename = "data/googlebooks-eng-all-1gram-20120701-a.gz"

for line in gzip.open(filename, "rt"):
    fields = line.lower().strip().split()

    line_count += 1

    if line_count % 100000 == 0:
        print(filename, line_count)
        break

    count = int(fields[2])
    word = fields[0]

    if "_" in word:
        continue

    letters = [char for char in word if char in characters]

    if len(letters) != len(word):
        continue

    for letter in letters:
        if letter not in dict_char:
            continue

        counts[dict_char[letter]] += count

total = np.sum(counts)
list_char = list(dict_char.items())
list_char.sort(key=lambda x: x[1])

for key, value in enumerate(list_char):
    print(value[0], counts[key]/total)


Not surprisingly, the most common character is the letter *a*. This is an artifact of the fact that we are using the datafile containing only words that start with the letter *a*. If you were to run it on the entire dataset, the resutls shown in the slides would be found. 

For simplicity, I've also included the complete table for all 5 languages in the repository. This is the datset that we will used to build our language detector.

## Visualization

Let's start by making a quick visualization of the probabiltity distributions for each language. THe first step is to load up the language character frequency from the file:

In [30]:
P_letter_lang = pd.read_csv('data/table_langs.dat', sep=' ', header=0, index_col = 0)

The histogram for the English language character frequency is:

In [31]:
plt.bar(range(26), P_letter_lang["eng"])
plt.xticks(list(range(26)), P_letter_lang.index)
plt.xlabel("letter")
plt.ylabel("P(letter)")

But we can easily find significant differences between the various languages

In [32]:
ax = P_letter_lang.plot()
ax.set_xticks(list(range(26)))
ax.set_xticklabels(P_letter_lang.index)
ax.legend(["English", "French", "German", "Italian", "Spanish"])
ax.set_xlabel("letter")
ax.set_ylabel("P(letter)")

As we can see, there definitely some common trends (the letters *q* and *j* are underrepreented across all languages), there are also some significant peaks that will help us discriminate between one language and the next.

## The detector

Based on this table of data it is extremely simple to build a Naive Bayes classifier. To do so, one must just calculte the correct set of log likelihoods so that we may use them later on.

In [33]:
def process_data(P_letter_lang):
    langs = list(P_letter_lang.columns)

    P_letter = P_letter_lang.mean(axis=1)
    P_letter /= P_letter.sum()

    P_lang_letter = np.array(P_letter_lang)/(P_letter_lang.shape[1]*P_letter.values.reshape(-1, 1))

    L_lang_letter = np.log(P_lang_letter.T)

    return langs, P_letter, L_lang_letter

In [34]:
langs, P_letter, L_lang_letter = process_data(P_letter_lang)

Finally, we have all the tools we need to write down our mini detector:

In [35]:
def detect_lang(langs, P_letter, L_lang_letter, text):
    counts = np.zeros(26, dtype='int')
    pos = dict(zip(P_letter.index, range(26)))

    text_counts =  Counter(text).items()

    for letter, count in text_counts:
        if letter in pos:
            counts[pos[letter]] += count

    L_text = np.dot(L_lang_letter, counts)
    index = np.argmax(L_text)
    lang_text = langs[index]
    prob = np.exp(L_text[index])/np.sum(np.exp(L_text))*100

    return lang_text, prob, L_text

And that's all there is to it. So now let's test our detector with a few past headlines from Google News:

In [36]:
texts = {}
texts["eng"] = "North Korea’s Test of Nuclear Bomb Amplifies a Global Crisis".lower()
texts["ita"] = "Nucleare, Onu riunisce consiglio sicurezza. E Seul simula attacco alle basi di Kim".lower()
texts["fre"] = "Corée du Nord : les Etats-Unis prêts à utiliser leurs capacités nucléaires".lower()
texts["spa"] = "Estados Unidos amenaza con una “respuesta militar masiva” a Corea del Norte".lower()
texts["ger"] = "Überraschung".lower()
texts["ita2"] = "Wales lancia la Wikipedia delle news. Contro il fake in campo anche Google".lower()

for lang in texts:
    text = texts[lang]
    lang_text, prob, L_text = detect_lang(langs, P_letter, L_lang_letter, text)
    print(lang, lang_text, prob, text)

Overall we do a pretty good job. We get 5/6 correct and the only one we are missing is a specific case where there are a surprising number of English words in the middle of an Italian headline. 

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>